In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('binance_api_15min_RSI-BB-MACD.csv')

In [3]:
df

,timestamp,open,high,low,close,Sell,Stable,Buy,In/Out,change,rsi,macd,macd_signal,macd_histogram,bb_upper,bb_middle,bb_lower
0,2023-03-01 13:15:00,23685.87,23709.67,23636.50,23646.54,0,1,0,0,-0.166325,42.217013,14.063045,36.820802,-22.757757,23850.827189,23753.9030,23656.978811
1,2023-03-01 13:30:00,23647.11,23710.43,23616.45,23668.02,0,1,0,0,0.090755,45.298946,8.642122,31.185066,-22.542944,23853.257598,23749.4055,23645.553402
2,2023-03-01 13:45:00,23668.02,23703.48,23653.16,23689.45,0,1,0,0,0.090462,48.263717,6.005991,26.149251,-20.143261,23832.051384,23740.4890,23648.926616
3,2023-03-01 14:00:00,23689.97,23711.00,23651.09,23688.07,0,1,0,0,-0.005826,48.082988,3.762115,21.671824,-17.909709,23810.147526,23732.1890,23654.230474
4,2023-03-01 14:15:00,23688.07,23703.14,23663.00,23685.29,0,1,0,0,-0.011737,47.695520,1.739454,17.685350,-15.945896,23803.109309,23727.0465,23650.983691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17602,2023-08-31 23:00:00,26053.95,26057.99,26006.13,26009.09,0,0,0,0,-0.172517,35.038200,-166.958293,-197.639517,30.681223,26339.874748,26094.1355,25848.396252
17603,2023-08-31 23:15:00,26009.10,26016.84,25950.12,25954.11,0,0,0,0,-0.211835,32.574119,-161.991013,-190.509816,28.518803,26308.090792,26076.3345,25844.578208
17604,2023-08-31 23:30:00,25954.11,25979.99,25951.43,25976.24,0,0,0,0,0.085193,34.568737,-154.487861,-183.305425,28.817564,26282.081487,26062.2500,25842.418513
17605,2023-08-31 23:45:00,25976.23,25976.24,25912.01,25940.78,0,0,0,0,-0.136696,32.889793,-149.677496,-176.579839,26.902343,26243.331833,26044.6775,25846.023167


In [4]:
for i in range(len(df)):
    if df.loc[i, 'change'] * 100 == 0: print(f"There is a time period (index {i}) with zero change!")

There is a time period (index 2201) with zero change!
There is a time period (index 2202) with zero change!
There is a time period (index 2203) with zero change!
There is a time period (index 2204) with zero change!
There is a time period (index 2205) with zero change!
There is a time period (index 4339) with zero change!
There is a time period (index 12530) with zero change!
There is a time period (index 13063) with zero change!
There is a time period (index 13564) with zero change!
There is a time period (index 13748) with zero change!
There is a time period (index 13906) with zero change!
There is a time period (index 14342) with zero change!
There is a time period (index 15783) with zero change!
There is a time period (index 15791) with zero change!
There is a time period (index 15885) with zero change!
There is a time period (index 16186) with zero change!


In [5]:
df = df[~(df['change'] == 0)].reset_index(drop=True)

In [6]:
# eğer bull bir önceki değerde 1 değilse ve şimdi 1 ise
# eğer bull bir önceki değerde 1 ise ve şimdi 

# df['investment'] = 100 #$
# for i in range(0, len(df)-1):
#     if df.loc[i, 'bull'] == 1:
#         df.loc[i+1, 'investment'] = (df.loc[i, 'investment'] * df.loc[i+1, 'change'] / 100) + df.loc[i-1, 'investment'] #inside
#     elif df.loc[i, 'volatile'] == 1:
#         if df.loc[i, 'investment'] != df.loc[i-1, 'investment']:
#             df.loc[i+1, 'investment'] = (df.loc[i, 'investment'] * df.loc[i+1, 'change'] / 100) + df.loc[i-1, 'investment'] #inside
#         else: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not inside
    
#     elif df.loc[i, 'bear'] == 1: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not inside

# df.loc[:,['bear', 'volatile', 'bull', 'change', 'investment']]
# print("The investment strategy profits {:.0f}%".format(df.iloc[-1, -1] - df.iloc[0, -1]))

In [7]:
# df['investment'] = 100 #$
# for i in range(0, len(df)-1):
#     if df.loc[i, 'bull'] == 1:
#         df.loc[i+1, 'investment'] = (df.loc[i, 'investment'] * df.loc[i+1, 'change'] / 100) + df.loc[i-1, 'investment'] #inside
#     elif df.loc[i, 'volatile'] == 1: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not insid
  
#     elif df.loc[i, 'bear'] == 1: df.loc[i+1, 'investment'] = df.loc[i, 'investment'] #not inside

# df.loc[:,['bear', 'volatile', 'bull', 'change', 'investment']]
# print("The investment strategy profits {:.0f}%".format(df.iloc[-1, -1] - df.iloc[0, -1]))

In [8]:
df

,timestamp,open,high,low,close,Sell,Stable,Buy,In/Out,change,rsi,macd,macd_signal,macd_histogram,bb_upper,bb_middle,bb_lower
0,2023-03-01 13:15:00,23685.87,23709.67,23636.50,23646.54,0,1,0,0,-0.166325,42.217013,14.063045,36.820802,-22.757757,23850.827189,23753.9030,23656.978811
1,2023-03-01 13:30:00,23647.11,23710.43,23616.45,23668.02,0,1,0,0,0.090755,45.298946,8.642122,31.185066,-22.542944,23853.257598,23749.4055,23645.553402
2,2023-03-01 13:45:00,23668.02,23703.48,23653.16,23689.45,0,1,0,0,0.090462,48.263717,6.005991,26.149251,-20.143261,23832.051384,23740.4890,23648.926616
3,2023-03-01 14:00:00,23689.97,23711.00,23651.09,23688.07,0,1,0,0,-0.005826,48.082988,3.762115,21.671824,-17.909709,23810.147526,23732.1890,23654.230474
4,2023-03-01 14:15:00,23688.07,23703.14,23663.00,23685.29,0,1,0,0,-0.011737,47.695520,1.739454,17.685350,-15.945896,23803.109309,23727.0465,23650.983691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17586,2023-08-31 23:00:00,26053.95,26057.99,26006.13,26009.09,0,0,0,0,-0.172517,35.038200,-166.958293,-197.639517,30.681223,26339.874748,26094.1355,25848.396252
17587,2023-08-31 23:15:00,26009.10,26016.84,25950.12,25954.11,0,0,0,0,-0.211835,32.574119,-161.991013,-190.509816,28.518803,26308.090792,26076.3345,25844.578208
17588,2023-08-31 23:30:00,25954.11,25979.99,25951.43,25976.24,0,0,0,0,0.085193,34.568737,-154.487861,-183.305425,28.817564,26282.081487,26062.2500,25842.418513
17589,2023-08-31 23:45:00,25976.23,25976.24,25912.01,25940.78,0,0,0,0,-0.136696,32.889793,-149.677496,-176.579839,26.902343,26243.331833,26044.6775,25846.023167


In [9]:
df.columns.to_series(index=range(len(df.columns)))

0          timestamp
1               open
2               high
3                low
4              close
5               Sell
6             Stable
7                Buy
8             In/Out
9             change
10               rsi
11              macd
12       macd_signal
13    macd_histogram
14          bb_upper
15         bb_middle
16          bb_lower
dtype: object

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_array = scaler.fit_transform(df.iloc[:,[4,10,11,12,13,14,15,16]])

scaled_df_X = pd.DataFrame(data = scaled_array, columns=[
 'close',
 'rsi',
 'macd',
 'macd_signal',
 'macd_histogram',
 'bb_upper',
 'bb_middle',
 'bb_lower'])

change_X = df.loc[:,['change']]

change_X

,change
0,-0.166325
1,0.090755
2,0.090462
3,-0.005826
4,-0.011737
...,...
17586,-0.172517
17587,-0.211835
17588,0.085193
17589,-0.136696


In [11]:
x = pd.DataFrame(scaled_df_X)

In [12]:
x

,close,rsi,macd,macd_signal,macd_histogram,bb_upper,bb_middle,bb_lower
0,-1.868929,-0.693754,0.182979,0.533504,-1.064028,-1.852019,-1.821633,-1.781604
1,-1.859358,-0.414541,0.106762,0.449334,-1.053962,-1.850934,-1.823638,-1.786663
2,-1.849809,-0.145943,0.069699,0.374123,-0.941516,-1.860403,-1.827614,-1.785169
3,-1.850424,-0.162316,0.038151,0.307252,-0.836854,-1.870183,-1.831314,-1.782821
4,-1.851662,-0.197420,0.009713,0.247714,-0.744832,-1.873326,-1.833607,-1.784258
...,...,...,...,...,...,...,...,...
17586,-0.816226,-1.344130,-2.362132,-2.968188,1.440067,-0.740604,-0.778249,-0.811348
17587,-0.840724,-1.567367,-2.292293,-2.861705,1.338738,-0.754796,-0.786186,-0.813038
17588,-0.830864,-1.386661,-2.186801,-2.754107,1.352738,-0.766410,-0.792466,-0.813995
17589,-0.846664,-1.538768,-2.119168,-2.653659,1.262992,-0.783712,-0.800300,-0.812399


In [13]:
target = df.loc[:,['In/Out']]
target.reset_index(drop=True, inplace=True)
target

,In/Out
0,0
1,0
2,0
3,0
4,0
...,...
17586,0
17587,0
17588,0
17589,0


In [14]:
target.value_counts()

In/Out
0         11015
1          6576
dtype: int64

**How does labeled data perform?** (for train set)

In [15]:
x.values

array([[-1.86892855, -0.69375365,  0.18297893, ..., -1.85201896,
        -1.82163314, -1.78160414],
       [-1.85935752, -0.41454103,  0.10676223, ..., -1.85093373,
        -1.82363833, -1.78666277],
       [-1.84980877, -0.14594298,  0.06969893, ..., -1.86040277,
        -1.82761372, -1.78516927],
       ...,
       [-0.8308637 , -1.38666121, -2.18680087, ..., -0.76640982,
        -0.79246551, -0.81399458],
       [-0.84666392, -1.53876782, -2.11916845, ..., -0.7837124 ,
        -0.80030014, -0.81239861],
       [-0.82441618, -1.12170759, -1.98619786, ..., -0.78831763,
        -0.8035031 , -0.81419372]])

In [16]:
df.change.values

array([-0.16632454,  0.09075537,  0.09046221, ...,  0.08519324,
       -0.13669597,  0.1921071 ])

In [17]:
from myCombinedLSTMClassifier import My_Combined_LSTM_Classifier

In [18]:
myObject1 = My_Combined_LSTM_Classifier(
    x = x.values,
    x_change = df.change.values,
    y = target.values,

    test_num=3000,
    time_steps=60,
    model_activation = 'relu',
    dropout=0.4,
    hidden_layer_num=3,
    epoch = 1,
    fit_verbose = 1,
    batch_size=32
)

94/94 [==============================] - 8s 66ms/step


c:\Users\anerg\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
[[[ 0.72499487  1.37074482  0.69352338 ...  0.95062694  0.69334324
    0.65716514]
  [ 0.72195602  1.27529234  0.71427743 ...  0.81631294  0.70021308
    0.66296761]
  [ 0.72344426  1.30297336  0.72597585 ...  0.6847169   0.70639125
    0.6685554 ]
  ...
  [ 0.62784979 -1.28389446 -0.59399372 ... -0.63694311  0.66987269
    0.67158224]
  [ 0.62006107 -1.4688516  -0.61500202 ... -0.56509322  0.66823722
    0.66771498]
  [ 0.61716035 -1.53722324 -0.63192241 ... -0.49671378  0.6658945
    0.6636959 ]]

 [[ 0.72195602  1.27529234  0.71427743 ...  0.81631294  0.70021308
    0.66296761]
  [ 0.72344426  1.30297336  0.72597585 ...  0.6847169   0.70639125
    0.6685554 ]
  [ 0.70745689  0.7831245   0.68645796 ...  0.44288307  0.70747779
    0.67439197]
  ...
  [ 0.62006107 -1.4688516  -0.61500202 ... -0.56509322  0.66823722
    0.66771498]
  [ 0.61716035 -1.53722324 -0.63192241 ... -0.49671378  0.6658945
    0.6636959 ]
  [ 0.60004122 -1.90904617 -0.68123662 

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
model_tune_object = My_Combined_LSTM_Classifier(
    x = x.values,
    x_change = df.change.values,
    y = target.values,

    test_num=3000,
    time_steps=3,
    model_activation = 'relu',
    dropout=0.4,
    hidden_layer_num=2,
    epoch = 1,
    fit_verbose = 0,
    batch_size=32
)

NameError: name 'My_Combined_LSTM_Classifier' is not defined

In [24]:
pd.set_option('display.width', 500)

model_tune_df = model_tune_object.model_tune(
    test_num = [3500], 
    epoch = [20,30], 
    time_steps = [24,48], 
    units = [30,45],
    dropout = [0.3, 0.5], 
    hidden_layer_num = [3,5], 
    model_activation = ['relu'], 
    batch_size = [24,48]
)
model_tune_df

THE MODEL IS TUNING!
SEE U AN ETERNITY LATER *_*

Total Progress is creating by 64 steps.

Tune Step: 1/64 is done! It took 6 min 39 sec
Tune Step: 2/64 is done! It took 3 min 49 sec
Tune Step: 3/64 is done! It took 11 min 1 sec
Tune Step: 4/64 is done! It took 5 min 45 sec
Tune Step: 5/64 is done! It took 7 min 4 sec
Tune Step: 6/64 is done! It took 3 min 59 sec
Tune Step: 7/64 is done! It took 10 min 43 sec
Tune Step: 8/64 is done! It took 6 min 34 sec
Tune Step: 9/64 is done! It took 7 min 20 sec
Tune Step: 10/64 is done! It took 4 min 47 sec
Tune Step: 11/64 is done! It took 11 min 35 sec
Tune Step: 12/64 is done! It took 7 min 37 sec
Tune Step: 13/64 is done! It took 8 min 2 sec
Tune Step: 14/64 is done! It took 5 min 8 sec
Tune Step: 15/64 is done! It took 12 min 8 sec
Tune Step: 16/64 is done! It took 9 min 49 sec
Tune Step: 17/64 is done! It took 15 min 24 sec
Tune Step: 18/64 is done! It took 10 min 9 sec


c:\Users\anerg\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tune Step: 19/64 is done! It took 18 min 28 sec
Tune Step: 20/64 is done! It took 11 min 43 sec
Tune Step: 21/64 is done! It took 11 min 31 sec
Tune Step: 22/64 is done! It took 7 min 45 sec


c:\Users\anerg\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tune Step: 23/64 is done! It took 21 min 20 sec
Tune Step: 24/64 is done! It took 13 min 54 sec
Tune Step: 25/64 is done! It took 17 min 55 sec
Tune Step: 26/64 is done! It took 11 min 29 sec
Tune Step: 27/64 is done! It took 26 min 38 sec


...

c:\Users\anerg\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tune Step: 60/64 is done! It took 21 min 45 sec
Tune Step: 61/64 is done! It took 22 min 12 sec
Tune Step: 62/64 is done! It took 14 min 44 sec
Tune Step: 63/64 is done! It took 33 min 21 sec


c:\Users\anerg\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Tune Step: 64/64 is done! It took 22 min 16 sec


,profit,train_precision,test_precision,train_recall,test_recall,test_num,epoch,time_steps,units,dropout,hidden_layer_num,model_activation,batch_size
55,4,53,35,36,34,3500,30,48,30,0.5,5,relu,48
30,4,55,32,18,27,3500,20,48,45,0.5,5,relu,24
0,2,70,25,58,35,3500,20,24,30,0.3,3,relu,24
16,1,69,26,63,42,3500,20,48,30,0.3,3,relu,24
59,0,27,0,0,0,3500,30,48,45,0.3,5,relu,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,-10,60,27,65,54,3500,30,24,30,0.5,5,relu,48
58,-1,52,10,9,2,3500,30,48,45,0.3,5,relu,24
12,-1,69,19,61,28,3500,20,24,45,0.5,3,relu,24
62,-1,55,19,8,11,3500,30,48,45,0.5,5,relu,24


In [30]:
model_tune_df.sort_values(by='profit', ascending=False).iloc[:10,:]

,profit,train_precision,test_precision,train_recall,test_recall,test_num,epoch,time_steps,units,dropout,hidden_layer_num,model_activation,batch_size
55,4,53,35,36,34,3500,30,48,30,0.5,5,relu,48
30,4,55,32,18,27,3500,20,48,45,0.5,5,relu,24
0,2,70,25,58,35,3500,20,24,30,0.3,3,relu,24
16,1,69,26,63,42,3500,20,48,30,0.3,3,relu,24
59,0,27,0,0,0,3500,30,48,45,0.3,5,relu,48
57,0,38,0,0,0,3500,30,48,45,0.3,3,relu,48
56,0,42,20,6,3,3500,30,48,45,0.3,3,relu,24
51,0,35,0,0,0,3500,30,48,30,0.3,5,relu,48
50,0,54,43,20,13,3500,30,48,30,0.3,5,relu,24
41,0,78,21,72,35,3500,30,24,45,0.3,3,relu,48


### binance_api_15min_RSI-BB-MACD.csv (**15/0005/00025**)

![Alt text](image-3.png)

### binance_api_mediumtune_15min_sRSI-BB.csv data results

![](image-2.png)